In [4]:

from network_evaluation_tools import data_import_tools as dit
from network_evaluation_tools import network_evaluation_functions as nef
from network_evaluation_tools import network_propagation as prop
import pandas as pd
import numpy as np
import networkx as nx

SyntaxError: invalid syntax (data_import_tools.py, line 17)

In [2]:
cd ..

/home/karen/Documents/GitHub/Network_Evaluation_Tools


In [3]:
evaluation_network_folder = 'Data/Networks/'
evaluation_network_folder = 'Data/NetworkCYJS/'
evaluation_lists_folder = 'Data/Evaluations/'

In [5]:
from os import listdir, path
from os.path import isfile, join


In [5]:
list_networks = [evaluation_network_folder+f for f in listdir(evaluation_network_folder) if isfile(join(evaluation_network_folder, f))]

In [ ]:
#list_evaluations

In [6]:
list_evaluations = [evaluation_lists_folder+f for f in listdir(evaluation_lists_folder) if isfile(join(evaluation_lists_folder, f))]

NameError: name 'evaluation_lists_folder' is not defined

In [ ]:
# network_name = path.splitext(evaluating_file)[0].split(evaluation_lists_folder)[1] +"_" + path.splitext(evaluating_network)[0].split(evaluation_network_folder)[1]

In [7]:
%%capture
performances = {}
gains = {}
for evaluating_file in list_evaluations:
    genesets = dit.load_node_sets(evaluating_file)
    for evaluating_network in list_networks:
        network_name = path.splitext(evaluating_file)[0].split(evaluation_lists_folder)[1] +"_" + path.splitext(evaluating_network)[0].split(evaluation_network_folder)[1]
        network = nx.read_gpickle(evaluating_network)
        genesets_p = nef.calculate_p(network, genesets)
        network_nodes = [str(gene) for gene in network.nodes()]
        nodesets_p = {}
        selected = {}
        m=-0.18887257
        b=0.64897403
        nodesets = genesets
        for nodeset in nodesets:
                nodesets_coverage = len([node for node in nodesets[nodeset] if node in network_nodes])
                if nodesets_coverage > 0:
                    nodesets_p[nodeset] = round(m*np.log10(nodesets_coverage)+b, 4)
                    selected[nodeset] = nodesets[nodeset]
                else:
                    print (nodeset, ' does not have common genes')
        genesets_p = nodesets_p
        genesets = selected
        alpha = prop.calculate_alpha(network)
        kernel = nef.construct_prop_kernel(network, alpha=alpha, verbose=True)
        AUPRC_values = nef.small_network_AUPRC_wrapper(kernel, genesets, genesets_p, n=30, cores=1, verbose=True)
        null_AUPRCs = []
        for i in range(10):
            shuffNet = nef.shuffle_network(network, max_tries_n=10, verbose=True)
            shuffNet_kernel = nef.construct_prop_kernel(shuffNet, alpha=alpha, verbose=False)
            shuffNet_AUPRCs = nef.small_network_AUPRC_wrapper(shuffNet_kernel, genesets, genesets_p, n=30, cores=4, verbose=False)
            null_AUPRCs.append(shuffNet_AUPRCs)
            # # print 'shuffNet', repr(i+1), 'AUPRCs calculated'
        null_AUPRCs_table = pd.concat(null_AUPRCs, axis=1)
        null_AUPRCs_table.columns = ['shuffNet'+repr(i+1) for i in range(len(null_AUPRCs))]
        network_performance = nef.calculate_network_performance_score(AUPRC_values, null_AUPRCs_table, verbose=True)
        network_performance.name = network_name
        network_perf_gain = nef.calculate_network_performance_gain(AUPRC_values, null_AUPRCs_table, verbose=True)
        network_perf_gain.name = network_name
        performances[network_name]=network_performance
        gains[network_name]=network_performance

        

NameError: name 'list_evaluations' is not defined

In [ ]:
performances['elements']= section_lens
gains['elements']= section_lens

In [ ]:
df_p = pd.DataFrame(performances)
df_p.to_csv('Data/Network_Performance_J.csv')

In [ ]:
df_g = pd.DataFrame(performances)
df_g.to_csv('Data/Network_Performance_Gain_J.csv')

In [ ]:
pd.Series(performances['DisGeNET_genesets_Young2Old_95_edge_14']).plot(kind='hist', bins=[0,10,20]) # `bins` defines the start and end points of bins


In [ ]:
# Rank network on average performance across gene sets vs performance on same gene sets in previous network set
all_network_performance = pd.read_csv('Data/Network_Performance_J.csv', index_col=0, header=None)


In [ ]:
all_network_performance_filt = pd.concat([network_performance, all_network_performance.ix[network_performance.index]], axis=1)
network_performance_rank_table = all_network_performance_filt.rank(axis=1, ascending=False)
network_performance_rankings = network_performance_rank_table['Test Network']

In [ ]:
# Rank network on average performance gain across gene sets vs performance gain on same gene sets in previous network set
all_network_perf_gain = pd.read_csv('Data/Network_Performance_Gain_J.csv', index_col=0, header=None)
all_network_perf_gain_filt = pd.concat([network_perf_gain, all_network_perf_gain.ix[network_perf_gain.index]], axis=1)



In [ ]:
all_network_perf_gain_filt[:6]

In [ ]:
network_perf_gain_rank_table = all_network_performance_filt.rank(axis=1, ascending=False)



In [ ]:
network_perf_gain_rankings = network_perf_gain_rank_table['Test Network']

In [ ]:
network_perf_gain_rank_table

In [ ]:
# Network Performance
network_performance_metric_ranks = pd.concat([network_performance, network_performance_rankings, network_perf_gain, network_perf_gain_rankings], axis=1)
network_performance_metric_ranks.columns = ['Network Performance', 'Network Performance Rank', 'Network Performance Gain', 'Network Performance Gain Rank']
network_performance_metric_ranks.sort_values(by=['Network Performance Rank', 'Network Performance', 'Network Performance Gain Rank', 'Network Performance Gain'],
                                             ascending=[True, False, True, False])

In [ ]:
# Construct network summary table
network_summary = {}
network_summary['Nodes'] = int(len(network.nodes()))
network_summary['Edges'] = int(len(network.edges()))


In [ ]:
network_summary['Avg Node Degree'] = np.mean(dict(network.degree()).values())


In [ ]:
network.degree()

In [ ]:
network_summary['Edge Density'] = 2*network_summary['Edges'] / float((network_summary['Nodes']*(network_summary['Nodes']-1)))
network_summary['Avg Network Performance Rank'] = network_performance_rankings.mean()
network_summary['Avg Network Performance Rank, Rank'] = int(network_performance_rank_table.mean().rank().ix['Test Network'])
network_summary['Avg Network Performance Gain Rank'] = network_perf_gain_rankings.mean()
network_summary['Avg Network Performance Gain Rank, Rank'] = int(network_perf_gain_rank_table.mean().rank().ix['Test Network'])
for item in ['Nodes', 'Edges' ,'Avg Node Degree', 'Edge Density', 'Avg Network Performance Rank', 'Avg Network Performance Rank, Rank',
             'Avg Network Performance Gain Rank', 'Avg Network Performance Gain Rank, Rank']:
    # print item+':\t'+repr(network_summary[item])